In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import AutoModel, AutoTokenizer
import pandas as pd

import math
import tqdm

In [78]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [53]:
# load datasets
df = pd.read_csv('../data/hackathon_train.csv', encoding='cp949', index_col=0)
df.index = range(len(df))
df.tail()

,User_ID,Gender,Age,MBTI,Q_number,Answer
11515,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...
11516,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...
11517,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...
11518,240,0,40,ISTJ,47,<아니다> 저는 감정에 휘둘리는 편이 아닙니다. 이유는 감정을 감추고 밖으로 표현하...
11519,240,0,40,ISTJ,48,<아니다> 저는 상대방 잘못일 때 상대방의 체면을 살려주기 위해 노력하지 않습니다....


In [8]:
df2 = pd.read_excel('../data2/train_data.xlsx', index_col=0)
df2.index = range(len(df2))
df2.tail()

,User_ID,Gender,Age,MBTI,Q_number,Short_Answer,Long_Answer
7195,240,female,30,ISTP,56,그렇다,거래처에 가격 조정 때문에 3군데를 가야 하는 상황이었는데 이야기 잘 통하는 곳 2...
7196,240,female,30,ISTP,57,아니다,상대방과 논쟁을 불러드릴 주제에는 관심이 없습니다 괜히 싸움을 일으키기 싫습니다
7197,240,female,30,ISTP,58,아니다,나에게 온 기회를 포기할 수 없다 양보를 하게 되면 나에게 기회는 없어지니깐
7198,240,female,30,ISTP,59,아니다,마감 기한이 정해지면 그 일을 끝날 때까지 늦게까지 일을 하고 퇴근하곤 합니다
7199,240,female,30,ISTP,60,그렇다,일을 할 때는 항상 진행될 때 이외의 상황을 생각하여 대처할 준비를 한다


In [9]:
dq = pd.read_excel('../data/Question.xlsx', index_col=0)
dq.tail()

,index.1,Question,Positive,Negative
index,,,,
56,56,단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편인가요? 그러한 최근 경험은 어떤...,절차대로 일을 완수하는 편이에요.,단계를 건너뛰는 일도 종종 있어요.
57,57,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에 관심이 많나요? 최근의 사례를 말씀...,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에 관심이 많아요.,NaN
58,58,자신보다 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수도 있나요? 이...,저보다 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수도 있어요.,저보다 다른 사람에게 더 필요한 기회라 해도 기회는 붙잡아야 해요.
59,59,마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요.,마감 기한은 힘들어요.,마감 기한을 잘 지켜요.
60,60,일이 원하는 대로 진행될 것이라는 자신감이 있나요? 그렇게 된 계기나 이유가 있나요.,일이 원하는 대로 진행될 것이라는 자신감이 있어요.,일의 진행되는 것에 비관적이에요..


In [12]:
load_tuned = False
MBTI = ['IE', 'SN', 'TF', 'JP']

In [79]:
if load_tuned:
    state = torch.load('tuned/roberta/0.pt', map_location=torch.device('cpu'))
    model = AutoModel.from_pretrained("klue/roberta-base").to(device)
    model.load_state_dict(state)
    tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
else:
    model = AutoModel.from_pretrained("klue/roberta-large").to(device)
    tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [50]:
answer_map = {'그렇다':0, '그렇자':0,
              '중립':1, '중랍':1, '중간':1, '보통':1, '중립/모르겠다': 1,
              '어렵다':2, '아니다':2, '<아니다':2, '아니요':2, '아니오':2}
def combine(dfx, filenum):
    list = []
    for i in range(len(dfx)):
        if filenum == 1:
            div = dfx.Answer[i].index('>')+1
            short_answer = dfx.Answer[i][:div].replace(' ', '')[1:-1]
            long_answer = dfx.Answer[i][div:]
        elif filenum == 2:
            short_answer = dfx.Short_Answer[i]
            long_answer = dfx.Long_Answer[i]
        answer_code = answer_map[short_answer]
        q = dfx.Q_number[i]
        prefix = {0:dq.Positive[q], 1:'', 2:dq.Negative[q]}[answer_code]
        if type(prefix) == float:
            prefix = ''
        list.append(prefix+' '+long_answer)
    return list

In [51]:
def long_only(dfx, filenum):
    list = []
    for i in range(len(dfx)):
        if filenum == 1:
            div = dfx.Answer[i].index('>')+1
            long_answer = dfx.Answer[i][div:]
        elif filenum == 2:
            long_answer = dfx.Long_Answer[i]
        list.append(long_answer)
    return list

In [52]:
comb = combine(df, 1)+combine(df2, 2)
long_answers = long_only(df, 1)+long_only(df2, 2)

In [55]:
long_answers[-1]

'일을 할 때는 항상 진행될 때 이외의 상황을 생각하여 대처할 준비를 한다'

In [56]:
max_length = model.config.max_position_embeddings
tensor = tokenizer(comb, max_length=max_length, return_tensors='pt', padding=True)
tensor_long = tokenizer(long_answers, max_length=max_length, return_tensors='pt', padding=True)

C:\Program Files\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [57]:
print(tokenizer.decode(tensor.input_ids[0]))
print(tokenizer.decode(tensor_long.input_ids[0]))

[CLS] 이미 있던 친구들과만 지내요. 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만 지냅니다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [58]:
torch.save(tensor, 'roberta/tokens.pt')
torch.save(tensor_long, 'roberta/tokens_long.pt')

In [59]:
tensor = torch.load('roberta/tokens.pt').input_ids
tensor_long = torch.load('roberta/tokens_long.pt').input_ids

In [61]:
print(tokenizer.decode(tensor[-1]))
for x in tensor[0]:
    print(tokenizer.decode(x))

[CLS] 일이 원하는 대로 진행될 것이라는 자신감이 있어요. 일을 할 때는 항상 진행될 때 이외의 상황을 생각하여 대처할 준비를 한다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [62]:
torch.max(tensor)

tensor(31485)

In [63]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [74]:
ds = MyMapDataset(tensor)
dl = DataLoader(ds, batch_size=64, shuffle=False)

In [80]:
def forward(model, dl):
    pooled = []
    hidden = []
    model.eval()
    for data in tqdm.tqdm(dl):
        with torch.no_grad():
            data = data.to(device)
            output = model(data, output_hidden_states=True).cpu()
        p, h = output.pooler_output, output.last_hidden_state
        pooled.append(p)
        hidden.append(h[:,0,:]) # only [CLS] token embedding
    return torch.cat(pooled), torch.cat(hidden)

In [81]:
def forward_tuned(model, dl):
    embed_l = []
    model.eval()
    for data in tqdm.tqdm(dl):
        with torch.no_grad():
            data = data.to(device)
            embed = model(data).cpu()
        embed = embed.last_hidden_state[:,0,:]
        embed_l.append(embed)
    return torch.cat(embed_l)

In [82]:
forward_func = forward_tuned if load_tuned else forward
result = forward_func(model, dl)

  0%|          | 0/293 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [101]:
result.shape

torch.Size([11520, 768])

In [102]:
torch.save(result, 'roberta/tuned_embed.pt')